In [152]:
from util.utils import *
import os

In [153]:
## load datasets
def loadDataframes():
    cat62 = openDataframe("cat62_bronze.csv")
    espera = openDataframe("espera_bronze.csv")
    bimtra = openDataframe("bimtra_bronze.csv")
    metaf = openDataframe("metaf_bronze.csv")
    metar = openDataframe("metar_bronze.csv")
    tc_prev = openDataframe("tc-prev_bronze.csv")
    tc_real = openDataframe("tc-real_bronze.csv")
    return cat62, espera, bimtra, metaf, metar, tc_prev, tc_real

cat62, espera, bimtra, metaf, metar, tc_prev, tc_real = loadDataframes()

In [154]:
## column names of each dataframe
def getColumns():
    cat62_cols = cat62.columns
    espera_cols = espera.columns
    bimtra_cols = bimtra.columns
    metaf_cols = metaf.columns
    metar_cols = metar.columns
    tc_prev_cols = tc_prev.columns
    tc_real_cols = tc_real.columns
    return cat62_cols, espera_cols, bimtra_cols, metaf_cols, metar_cols, tc_prev_cols, tc_real_cols

cat62_cols, espera_cols, bimtra_cols, metaf_cols, metar_cols, tc_prev_cols, tc_real_cols = getColumns()
print(cat62_cols)
print(espera_cols)
print(bimtra_cols)
print(metaf_cols)
print(metar_cols)
print(tc_prev_cols)
print(tc_real_cols)

Index(['flightid', 'lat', 'lon', 'flightlevel', 'speed', 'dt_radar',
       'dt_radar_formatted'],
      dtype='object')
Index(['esperas', 'hora', 'aero', 'hora_formatted'], dtype='object')
Index(['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr', 'dt_dep_formatted',
       'dt_arr_formatted'],
      dtype='object')
Index(['hora', 'metaf', 'aero', 'hora_formatted'], dtype='object')
Index(['hora', 'metar', 'aero', 'hora_formatted'], dtype='object')
Index(['hora', 'troca', 'aero', 'hora_formatted'], dtype='object')
Index(['hora', 'nova_cabeceira', 'antiga_cabeceira', 'aero', 'hora_formatted'], dtype='object')


In [155]:
## merge cat62 and bimtra dataframes with flightid
def mergeWithFlightID(cat62, bimtra):
    cat62_bimtra = pd.merge(cat62, bimtra, on="flightid")
    return cat62_bimtra

merged_df = mergeWithFlightID(cat62, bimtra)
merged_df.head()

,flightid,lat,lon,flightlevel,speed,dt_radar,dt_radar_formatted,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted
0,ce43268d98c9e81575d0efeb92383aba,-0.360540,-0.768632,260.0,341.0,1677629941000,"Tuesday, February 28, 2023 21:19:01",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"
1,ce43268d98c9e81575d0efeb92383aba,-0.370018,-0.770168,260.0,333.0,1677630303000,"Tuesday, February 28, 2023 21:25:03",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"
2,ce43268d98c9e81575d0efeb92383aba,-0.377761,-0.771272,260.0,349.0,1677630601000,"Tuesday, February 28, 2023 21:30:01",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"
3,ce43268d98c9e81575d0efeb92383aba,-0.393593,-0.769379,92.0,291.0,1677631744000,"Tuesday, February 28, 2023 21:49:04",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"
4,ce43268d98c9e81575d0efeb92383aba,-0.400201,-0.768154,90.0,255.0,1677632283000,"Tuesday, February 28, 2023 21:58:03",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"


In [156]:
## crie uma função que pega a coluna data_dep e "arredonda" para a hora atual inteira
## exemplo: se data_dep = 2019-01-01 12:34:56, a função deve retornar 2019-01-01 12:00:00
## a hora em dt_dep esta no seguinte formato: 1677629154000, 1677629154000, 1677629154000, etc
def roundHour(dt_dep):
    dt_dep = dt_dep/1000
    dt_dep = datetime.fromtimestamp(dt_dep)
    dt_dep = dt_dep.replace(minute=0, second=0, microsecond=0)
    return dt_dep


merged_df['hora_ref'] = merged_df['dt_dep'].apply(roundHour)
merged_df.head()

,flightid,lat,lon,flightlevel,speed,dt_radar,dt_radar_formatted,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted,hora_ref
0,ce43268d98c9e81575d0efeb92383aba,-0.360540,-0.768632,260.0,341.0,1677629941000,"Tuesday, February 28, 2023 21:19:01",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00
1,ce43268d98c9e81575d0efeb92383aba,-0.370018,-0.770168,260.0,333.0,1677630303000,"Tuesday, February 28, 2023 21:25:03",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00
2,ce43268d98c9e81575d0efeb92383aba,-0.377761,-0.771272,260.0,349.0,1677630601000,"Tuesday, February 28, 2023 21:30:01",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00
3,ce43268d98c9e81575d0efeb92383aba,-0.393593,-0.769379,92.0,291.0,1677631744000,"Tuesday, February 28, 2023 21:49:04",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00
4,ce43268d98c9e81575d0efeb92383aba,-0.400201,-0.768154,90.0,255.0,1677632283000,"Tuesday, February 28, 2023 21:58:03",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00


In [157]:
def groupById(df):
    grouped = df.groupby(['flightid', 'hora_ref'])  # Agrupe por flightid e hora_ref
    return grouped

def createCoordinatesList(group):
    lat_lon_list = [(row['lat'], row['lon']) for _, row in group.iterrows()]
    return lat_lon_list

def formatDataframe(df):
    grouped = groupById(df)
    
    dados_final = {
        'flightid': [],
        'origem': [],
        'destino': [],
        'dt_dep': [],
        'hora_ref': [],
        'snapshot_radar': [],
        'dt_arr': []
    }
    
    for (flight_id, hora_ref), group in grouped:  # Altere aqui para iterar corretamente sobre os grupos
        lat_lon_list = createCoordinatesList(group)
        
        origem = group['origem'].iloc[0]
        destino = group['destino'].iloc[0]
        dt_dep = group['dt_dep'].iloc[0]
        dt_arr = group['dt_arr'].iloc[0]
        
        dados_final['flightid'].append(flight_id)
        dados_final['origem'].append(origem)
        dados_final['destino'].append(destino)
        dados_final['dt_dep'].append(dt_dep)
        dados_final['hora_ref'].append(hora_ref)
        dados_final['snapshot_radar'].append(lat_lon_list)
        dados_final['dt_arr'].append(dt_arr)
    
    new_df = pd.DataFrame(dados_final)
    return new_df

formated_df = formatDataframe(merged_df)
formated_df.head()

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,dt_arr
0,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",1679145598000
1,0001d7d56ab8aaa1fd7e8fb4ae079f60,SBBR,SBCT,1677700968000,2023-03-01 17:00:00,"[(-0.345388457, -0.8614051214), (-0.3590661267...",1677706298000
2,0004796cee4930d622b6eabdc22cdb5b,SBPA,SBBR,1677832349000,2023-03-03 05:00:00,"[(-0.4760178852, -0.8794155068), (-0.478144077...",1677840097000
3,000897ae4e502951775f08651ea4d663,SBSP,SBBR,1680137219000,2023-03-29 21:00:00,"[(-0.4128358742, -0.817906891), (-0.4127446866...",1680142257000
4,000988161c4bb5bd05e0dfba694085b4,SBPA,SBFL,1679014304000,2023-03-16 21:00:00,"[(-0.5234733354, -0.8926822638), (-0.521382814...",1679016795000


In [158]:
len(formated_df)

26792

In [159]:
espera.head()

,esperas,hora,aero,hora_formatted
0,0,1677628800000,SBBR,"Tuesday, February 28, 2023 21:00:00"
1,0,1677632400000,SBBR,"Tuesday, February 28, 2023 22:00:00"
2,0,1677636000000,SBBR,"Tuesday, February 28, 2023 23:00:00"
3,0,1677639600000,SBBR,"Wednesday, March 01, 2023 00:00:00"
4,0,1677643200000,SBBR,"Wednesday, March 01, 2023 01:00:00"


In [160]:
formated_df.columns

Index(['flightid', 'origem', 'destino', 'dt_dep', 'hora_ref', 'snapshot_radar',
       'dt_arr'],
      dtype='object')

In [161]:
## merge espera with hora_ref
def mergeByRefDate(df, espera):
    merged_df = pd.merge(df, espera, on=['hora_ref'])
    ## drop 'hora' column
    merged_df = merged_df.drop(columns=['hora'])
    return merged_df

espera['hora_ref'] = espera['hora'].apply(roundHour)
formated_df = mergeByRefDate(formated_df, espera)
formated_df.head()

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,dt_arr,esperas,aero,hora_formatted
0,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",1679145598000,0,SBBR,"Saturday, March 18, 2023 08:00:00"
1,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",1679145598000,0,SBCF,"Saturday, March 18, 2023 08:00:00"
2,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",1679145598000,0,SBCT,"Saturday, March 18, 2023 08:00:00"
3,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",1679145598000,0,SBFL,"Saturday, March 18, 2023 08:00:00"
4,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",1679145598000,0,SBGL,"Saturday, March 18, 2023 08:00:00"


In [162]:
formated_df.hora_ref.unique()

array(['2023-03-18T08:00:00.000000000', '2023-03-01T17:00:00.000000000',
       '2023-03-03T05:00:00.000000000', '2023-03-29T21:00:00.000000000',
       '2023-03-16T21:00:00.000000000', '2023-03-11T02:00:00.000000000',
       '2023-03-03T11:00:00.000000000', '2023-03-30T06:00:00.000000000',
       '2023-03-02T08:00:00.000000000', '2023-03-26T11:00:00.000000000',
       '2023-03-30T15:00:00.000000000', '2023-03-09T11:00:00.000000000',
       '2023-03-21T12:00:00.000000000', '2023-03-28T10:00:00.000000000',
       '2023-03-27T08:00:00.000000000', '2023-03-30T18:00:00.000000000',
       '2023-03-10T06:00:00.000000000', '2023-03-12T21:00:00.000000000',
       '2023-03-20T21:00:00.000000000', '2023-03-05T22:00:00.000000000',
       '2023-03-20T11:00:00.000000000', '2023-03-22T23:00:00.000000000',
       '2023-03-12T14:00:00.000000000', '2023-03-09T14:00:00.000000000',
       '2023-03-09T17:00:00.000000000', '2023-03-12T17:00:00.000000000',
       '2023-03-08T15:00:00.000000000', '2023-03-01

In [163]:
def convertHour(hora_ref):
    # Formate a hora_ref para corresponder ao formato no nome do arquivo
    hora_formatada_str = hora_ref.strftime('%Y%m%d%H%M')
    hora_formatada = datetime.strptime(hora_formatada_str, '%Y%m%d%H%M')
    return hora_formatada


def findImageByHour(imagesDir, refHour):
    # Converter a hora_ref para o formato desejado
    formatedDate = refHour.strftime('%Y%m%d')
    convertedHour = convertHour(refHour)
    
    # Construir o nome do arquivo com base na data_formatada
    fileName = f'S11635384_{convertedHour}.jpg'
    
    # Construir o caminho completo do arquivo
    filePath = os.path.join(imagesDir, fileName)
    
    # Verificar se o arquivo existe
    if os.path.isfile(filePath):
        return filePath
    else:
        return None  # Arquivo não encontrado

imagesDir = '../images/'
#formated_df['path'] = formated_df['hora_ref'].apply(lambda x: findImageByHour(imagesDir, x))
#formated_df.head()

In [164]:
preprocessed_metar = pd.read_csv('./preprocessed_metar.csv')
preprocessed_metar.columns

Index(['hora', 'metaf', 'station', 'type', 'metar_hora_formated', 'station.1',
       'dew_point', 'windDirection', 'windVelocity', 'visibility', 'pressure',
       'weather', 'sky', 'skyFeet'],
      dtype='object')

In [165]:
preprocessed_metar['hora_ref'] = preprocessed_metar['hora'].apply(roundHour)
preprocessed_metar.columns

Index(['hora', 'metaf', 'station', 'type', 'metar_hora_formated', 'station.1',
       'dew_point', 'windDirection', 'windVelocity', 'visibility', 'pressure',
       'weather', 'sky', 'skyFeet', 'hora_ref'],
      dtype='object')

In [166]:
formated_df = mergeByRefDate(formated_df, preprocessed_metar)
formated_df

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,dt_arr,esperas,aero,hora_formatted,...,metar_hora_formated,station.1,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet
0,0001d7d56ab8aaa1fd7e8fb4ae079f60,SBBR,SBCT,1677700968000,2023-03-01 17:00:00,"[(-0.345388457, -0.8614051214), (-0.3590661267...",1677706298000,0,SBBR,"Wednesday, March 01, 2023 17:00:00",...,Fri Sep 01 17:00:00 2023,27.0,17.0,112.5,4,greater,1014.0,rain,a few clouds,3300
1,0001d7d56ab8aaa1fd7e8fb4ae079f60,SBBR,SBCT,1677700968000,2023-03-01 17:00:00,"[(-0.345388457, -0.8614051214), (-0.3590661267...",1677706298000,0,SBBR,"Wednesday, March 01, 2023 17:00:00",...,Fri Sep 01 17:00:00 2023,26.0,19.0,135.0,7,greater,1014.0,light thunderstorm with rain,scattered clouds,2500
2,0001d7d56ab8aaa1fd7e8fb4ae079f60,SBBR,SBCT,1677700968000,2023-03-01 17:00:00,"[(-0.345388457, -0.8614051214), (-0.3590661267...",1677706298000,0,SBCF,"Wednesday, March 01, 2023 17:00:00",...,Fri Sep 01 17:00:00 2023,27.0,17.0,112.5,4,greater,1014.0,rain,a few clouds,3300
3,0001d7d56ab8aaa1fd7e8fb4ae079f60,SBBR,SBCT,1677700968000,2023-03-01 17:00:00,"[(-0.345388457, -0.8614051214), (-0.3590661267...",1677706298000,0,SBCF,"Wednesday, March 01, 2023 17:00:00",...,Fri Sep 01 17:00:00 2023,26.0,19.0,135.0,7,greater,1014.0,light thunderstorm with rain,scattered clouds,2500
4,0001d7d56ab8aaa1fd7e8fb4ae079f60,SBBR,SBCT,1677700968000,2023-03-01 17:00:00,"[(-0.345388457, -0.8614051214), (-0.3590661267...",1677706298000,0,SBCT,"Wednesday, March 01, 2023 17:00:00",...,Fri Sep 01 17:00:00 2023,27.0,17.0,112.5,4,greater,1014.0,rain,a few clouds,3300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414703,ce618fc35d71bb2c0033d73118a6b431,SBSP,SBRJ,1677970978000,2023-03-04 20:00:00,"[(-0.4052998115, -0.7705120937), (-0.399194530...",1677973292000,0,SBRJ,"Saturday, March 04, 2023 20:00:00",...,Mon Sep 04 20:00:00 2023,22.0,19.0,67.5,8,4000,1014.0,thunderstorm with rain mist,broken clouds,1300
414704,ce618fc35d71bb2c0033d73118a6b431,SBSP,SBRJ,1677970978000,2023-03-04 20:00:00,"[(-0.4052998115, -0.7705120937), (-0.399194530...",1677973292000,0,SBSP,"Saturday, March 04, 2023 20:00:00",...,Mon Sep 04 20:00:00 2023,22.0,18.0,112.5,9,7000,1015.0,light thunderstorm with rain,scattered clouds,3300
414705,ce618fc35d71bb2c0033d73118a6b431,SBSP,SBRJ,1677970978000,2023-03-04 20:00:00,"[(-0.4052998115, -0.7705120937), (-0.399194530...",1677973292000,0,SBSP,"Saturday, March 04, 2023 20:00:00",...,Mon Sep 04 20:00:00 2023,22.0,19.0,67.5,8,4000,1014.0,thunderstorm with rain mist,broken clouds,1300
414706,ce618fc35d71bb2c0033d73118a6b431,SBSP,SBRJ,1677970978000,2023-03-04 20:00:00,"[(-0.4052998115, -0.7705120937), (-0.399194530...",1677973292000,0,SBSV,"Saturday, March 04, 2023 20:00:00",...,Mon Sep 04 20:00:00 2023,22.0,18.0,112.5,9,7000,1015.0,light thunderstorm with rain,scattered clouds,3300


In [167]:
# formated_df.path.unique()

In [168]:
def dropColumns(df, columns):
    df = df.drop(columns=columns, axis = 1)
    return df

formated_df = dropColumns(formated_df, ['flightid', 'hora_ref', 'snapshot_radar', 'hora_formatted'])
formated_df.head()

,origem,destino,dt_dep,dt_arr,esperas,aero,metaf,station,type,metar_hora_formated,station.1,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet
0,SBBR,SBCT,1677700968000,1677706298000,0,SBBR,METAF SBGR 012000Z 12004KT 9999 RA FEW033...,SBGR,"routine report, cycle 20 (automatic report)",Fri Sep 01 17:00:00 2023,27.0,17.0,112.5,4,greater,1014.0,rain,a few clouds,3300
1,SBBR,SBCT,1677700968000,1677706298000,0,SBBR,METAF SBSP 012000Z 13007KT 9999 -TSRA SCT02...,SBSP,"routine report, cycle 20 (automatic report)",Fri Sep 01 17:00:00 2023,26.0,19.0,135.0,7,greater,1014.0,light thunderstorm with rain,scattered clouds,2500
2,SBBR,SBCT,1677700968000,1677706298000,0,SBCF,METAF SBGR 012000Z 12004KT 9999 RA FEW033...,SBGR,"routine report, cycle 20 (automatic report)",Fri Sep 01 17:00:00 2023,27.0,17.0,112.5,4,greater,1014.0,rain,a few clouds,3300
3,SBBR,SBCT,1677700968000,1677706298000,0,SBCF,METAF SBSP 012000Z 13007KT 9999 -TSRA SCT02...,SBSP,"routine report, cycle 20 (automatic report)",Fri Sep 01 17:00:00 2023,26.0,19.0,135.0,7,greater,1014.0,light thunderstorm with rain,scattered clouds,2500
4,SBBR,SBCT,1677700968000,1677706298000,0,SBCT,METAF SBGR 012000Z 12004KT 9999 RA FEW033...,SBGR,"routine report, cycle 20 (automatic report)",Fri Sep 01 17:00:00 2023,27.0,17.0,112.5,4,greater,1014.0,rain,a few clouds,3300


In [169]:
formated_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414708 entries, 0 to 414707
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   origem               414708 non-null  object 
 1   destino              414708 non-null  object 
 2   dt_dep               414708 non-null  int64  
 3   dt_arr               414708 non-null  int64  
 4   esperas              414708 non-null  int64  
 5   aero                 414708 non-null  object 
 6   metaf                414708 non-null  object 
 7   station              414708 non-null  object 
 8   type                 414708 non-null  object 
 9   metar_hora_formated  414708 non-null  object 
 10  station.1            414708 non-null  float64
 11  dew_point            414708 non-null  float64
 12  windDirection        414708 non-null  float64
 13  windVelocity         414708 non-null  int64  
 14  visibility           414708 non-null  object 
 15  pressure         

In [170]:
## import onehot encode
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # Replace with your regression model of choice
from sklearn.ensemble import RandomForestRegressor

def encodeData(cat_features, num_features, scaler, encoder):
    ## Tratamento para variáveis categóricas
    cat_transformer = Pipeline(steps = [
        ('encoder', encoder)
    ])

    ## Tratamento para variáveis numéricas
    num_transformer = Pipeline(steps = [
        ('scaler', scaler)
    ])

    ## Transformando os dados
    preprocessor = ColumnTransformer(transformers = [
        ('numbers', num_transformer, num_features),
        ('categories', cat_transformer, cat_features)
    ])

    ## Retornando o processador de dados
    return preprocessor    

def getCategoricalVariables(df):
    # Use select_dtypes to filter columns of object dtype (categorical)
    categorical_cols = df.select_dtypes(include=['object', 'category'])
    return list(categorical_cols.columns)

def getNumericalVariables(df):
    # Use select_dtypes to filter columns of numerical dtype
    numerical_cols = df.select_dtypes(include=['number', 'float64', 'int64'])
    return list(numerical_cols.columns)


X = formated_df.drop(['dt_arr', 'metar_hora_formated', 'metaf', 'type'], axis = 1)
y = formated_df['dt_arr']


numericalVariables = getNumericalVariables(X)
categoricalVariables = getCategoricalVariables(X)

preprocessor = encodeData(categoricalVariables, numericalVariables, StandardScaler(), OneHotEncoder(handle_unknown='ignore'))

reg_model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])


# Define the number of folds for cross-validation
num_folds = 5

## Define the score method
scoring = 'neg_mean_squared_error'

# Create a k-fold cross-validation object
kf = KFold(n_splits=num_folds)

mse_scores = cross_val_score(reg_model, X, y, cv = kf, scoring = scoring)

# Calculate the mean and standard deviation of the MSE scores
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

# Print the results
print(f'MSE Scores: {mse_scores}')
print(f'Mean MSE: {mean_mse}')
print(f'Standard Deviation of MSE: {std_mse}')

MSE Scores: [-1.37881877e+12 -1.38548599e+12 -1.37281285e+12 -1.29982764e+12
 -1.47235788e+12]
Mean MSE: -1381860625438.4048
Standard Deviation of MSE: 54815004236.34347


In [171]:
# from sklearn.feature_selection import mutual_info_regression, SelectKBest, f_regression
# from sklearn.ensemble import RandomForestRegressor  # If you want to use Random Forest for regression
# import pandas as pd
# import numpy as np
# import plotly.graph_objs as go

# def createEstimatorScoreDF(x, estimatorScore, sortedIdx):
#     sortedCols = x.columns[sortedIdx][::-1]
#     estimatorScoreDF = pd.DataFrame(estimatorScore[sortedIdx][::-1], index=sortedCols, columns=['score'])
#     return estimatorScoreDF

# def plotEstimatorScore(x, sortedIdx, estimatorScore, title):
#     fig = go.Figure()
#     fig.add_trace(
#         go.Bar(x=x.columns[sortedIdx], y=estimatorScore[sortedIdx])
#     )
    
#     fig.update_layout(
#         xaxis=dict(title='Feature'),
#         yaxis=dict(title='Score'),
#         title=title,
#         xaxis_tickangle=-45,
#         margin=dict(l=50, r=50, t=50, b=50)
#     )
    
#     fig.show()

# def estimatorScoreRegression(X, y, title, preprocessor=None, algorithm='mutual_info_regression'):
#     np.random.seed(42)
#     if algorithm == 'mutual_info_regression':
#         estimatorScore = mutual_info_regression(X, y)
#     elif algorithm == 'SelectKBest':
#         selector = SelectKBest(score_func=f_regression, k=5)  # Example: Select top 10 features
#         X_transformed = preprocessor.fit_transform(X, y)
#         selector.fit(X_transformed, y)
#         estimatorScore = selector.scores_
#     elif algorithm == 'RandomForestImportance':
#         model = RandomForestRegressor()
#         X_transformed = preprocessor.fit_transform(X, y)
#         model.fit(X_transformed, y)
#         estimatorScore = model.feature_importances_
#     elif algorithm == 'LinearRegressionImportance':
#         model = LinearRegression()
#         X_transformed = preprocessor.fit_transform(X, y)
#         model.fit(X_transformed, y)
        
#         # Use absolute value of coefficients
#         estimatorScore = np.abs(model.coef_[0])

#     # Add more conditions for other feature importance algorithms here

#     sortedIdx = np.argsort(estimatorScore)
#     estimatorScoreDF = createEstimatorScoreDF(X, estimatorScore, sortedIdx)
#     plotEstimatorScore(X.columns, sortedIdx, estimatorScore, title)

#     return estimatorScoreDF


# no_categories_df = dropColumns(formated_df, getCategoricalVariables(formated_df))
# no_categories_df.head()

# X = no_categories_df.drop(['dt_arr'], axis = 1)
# y = no_categories_df['dt_arr']


# # Get feature importance scores for regression
# estimatorScores = estimatorScoreRegression(X, y, 'Feature Importance for Regression', preprocessor, algorithm='LinearRegressionImportance')
# estimatorScores


In [172]:
# formated_df.to_csv('formated_df.csv', index=False)